In [27]:
import gdown

gdown.download('https://bit.ly/3q9SZix', '20s_best_book.json', quiet=False)

Downloading...
From: https://bit.ly/3q9SZix
To: C:\Users\kibon\python_program\20s_best_book.json
100%|█████████████████████████████████████████████████████████████████████████████| 92.9k/92.9k [00:00<00:00, 1.82MB/s]


'20s_best_book.json'

In [28]:
import pandas as pd

books_df = pd.read_json('20s_best_book.json')
books_df.head()

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,461,https://image.aladin.co.kr/product/19359/16/co...
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,387,https://image.aladin.co.kr/product/24512/70/co...
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,03810,,813.7,문학 > 한국문학 > 소설,383,https://image.aladin.co.kr/product/19804/82/co...
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,03810,,813.7,문학 > 한국문학 > 소설,370,https://image.aladin.co.kr/product/24131/37/co...
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,03810,,813.7,문학 > 한국문학 > 소설,365,http://image.aladin.co.kr/product/16839/4/cove...


In [30]:
books = books_df[['no','ranking','bookname','authors','publisher','publication_year','isbn13']]
books.head()

,no,ranking,bookname,authors,publisher,publication_year,isbn13
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267


(1) 원하는 정보 추출 및 데이터 프레임으로 저장

In [64]:
mybooks = books_df[['bookname','authors','publication_year','isbn13']]
mybooks.head()

,bookname,authors,publication_year,isbn13
0,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,2019,9791190090018
1,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,2020,9791165341909
2,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,2019,9791188862290
3,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,2020,9788954672214
4,아몬드 :손원평 장편소설,지은이: 손원평,2017,9788936434267


In [65]:
import requests

isbn = 9791190090018      # '우리가 빛의 속도로 갈 수 없다면'의 ISBN
url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'

r = requests.get(url.format(isbn))

In [66]:
from bs4 import BeautifulSoup

In [67]:
soup = BeautifulSoup(r.text, 'html.parser')

In [68]:
prd_link = soup.find('a', attrs={'class':'gd_name'})

In [78]:
# '우리가 빛의 속도로 갈 수 없다면'의 상세 페이지 가져오기
url = 'http://www.yes24.com'+prd_link['href']
r = requests.get(url)

In [79]:
soup = BeautifulSoup(r.text, 'html.parser')
prd_detail = soup.find('div', attrs={'id':'infoset_specific'})

In [80]:
prd_tr_list = prd_detail.find_all('tr') #'tr' 로 묶여 있는 것을 찾음

In [81]:
for tr in prd_tr_list:
    if tr.find('th').get_text() == '쪽수, 무게, 크기':
        page_td = tr.find('td').get_text()
        break

In [82]:
def get_page_cnt(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 쪽수가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return tr.find('td').get_text().split()[0]
    return ''

In [83]:
get_page_cnt(9791190090018)

'330쪽'

In [84]:
#top10_books = books.head(10)
my_books = mybooks.head(10)

In [85]:
my_books

,bookname,authors,publication_year,isbn13
0,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,2019,9791190090018
1,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,2020,9791165341909
2,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,2019,9791188862290
3,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,2020,9788954672214
4,아몬드 :손원평 장편소설,지은이: 손원평,2017,9788936434267
5,피프티 피플 :정세랑 장편소설,지은이: 정세랑,2016,9788936434243
6,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,2020,9791165300005
7,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,2012,9788972756194
8,선량한 차별주의자,김지혜 지음,2019,9788936477196
9,쇼코의 미소 :최은영 소설,지은이: 최은영,2016,9788954641630


In [88]:
def get_page_cnt2(row):
    isbn = row['isbn13']
    return get_page_cnt(isbn)

In [87]:
page_count = my_books.apply(get_page_cnt2, axis=1)
print(page_count)
# 7번에는 isbn이 없어서 페이지가 안나왔음.

0    330쪽
1    300쪽
2    228쪽
3    340쪽
4    264쪽
5    396쪽
6    272쪽
7        
8    244쪽
9    296쪽
dtype: object


In [89]:
page_count = my_books.apply(lambda row: get_page_cnt(row['isbn13']), axis=1)

In [90]:
page_count.name = 'page_count'
print(page_count)

0    330쪽
1    300쪽
2    228쪽
3    340쪽
4    264쪽
5    396쪽
6    272쪽
7        
8    244쪽
9    296쪽
Name: page_count, dtype: object


In [91]:
mybooks_with_page_count = pd.merge(my_books, page_count,
                                 left_index=True, right_index=True)
mybooks_with_page_count

,bookname,authors,publication_year,isbn13,page_count
0,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,2019,9791190090018,330쪽
1,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,2020,9791165341909,300쪽
2,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,2019,9791188862290,228쪽
3,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,2020,9788954672214,340쪽
4,아몬드 :손원평 장편소설,지은이: 손원평,2017,9788936434267,264쪽
5,피프티 피플 :정세랑 장편소설,지은이: 정세랑,2016,9788936434243,396쪽
6,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,2020,9791165300005,272쪽
7,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,2012,9788972756194,
8,선량한 차별주의자,김지혜 지음,2019,9788936477196,244쪽
9,쇼코의 미소 :최은영 소설,지은이: 최은영,2016,9788954641630,296쪽
